In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd 
import mlflow
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
try:
    mlflow.set_experiment("lstm")
except mlflow.exceptions.MlflowException:
    mlflow.create_experiment("lstm")
    mlflow.set_experiment("lstm")

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sarthakagarwal/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
2025/04/07 12:30:38 INFO mlflow.tracking.fluent: Experiment with name 'lstm' does not exist. Creating a new experiment.


In [2]:
# data = gutenberg.raw('shakespeare-hamlet.txt')

In [ ]:
# with open("data/hamlet.txt", "w") as file:
#     file.write(data)

In [6]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [7]:
with open("data/hamlet.txt", "r") as file:
    text = file.read().lower()

In [8]:
# Index creation for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(total_words)


4818


In [11]:
# input sequence creation
input_sequence = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequence.append(n_gram_sequence)
    

In [15]:
# pad sequence 
max_sequence_length = max(len(i) for i in input_sequence)
input_sequence = np.array(
    pad_sequences(
        input_sequence, 
        maxlen=max_sequence_length,
        padding='pre'
    )
)

In [17]:
# create predictor and labels
import tensorflow as tf 

x, y = input_sequence[:, :-1], input_sequence[:, -1]

In [18]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42
)

In [39]:
# train our lstm model on the dataset
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


# model=Sequential()
# model.add(Embedding(total_words,100,input_length=max_sequence_length-1))
# model.add(LSTM(150,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dense(total_words,activation="softmax"))

# # #Compile the model
# model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
# model.summary()


callback = EarlyStopping(monitor='val_loss', patience=20,
                         restore_best_weights=True)

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [40]:
mlflow.autolog()
with mlflow.start_run(run_name="lstm"):
    history = model.fit(
        x_train,
        y_train,
        epochs=100,
        validation_data=(x_test, y_test),
        verbose=1,
        callbacks=[callback]
    )

2025/04/07 13:02:41 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.8.0, but the installed version is 3.9.2. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2025/04/07 13:02:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/04/07 13:02:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/04/07 13:02:41 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2025/04/07 13:02:41 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2025/04/07 13:02:41 WARNING mlflow.tensorflow: Encountered unexpected error while infer

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0295 - loss: 7.1368

644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 29ms/step - accuracy: 0.0296 - loss: 7.1364 - val_accuracy: 0.0365 - val_loss: 6.6881
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 27ms/step - accuracy: 0.0415 - loss: 6.4398 - val_accuracy: 0.0404 - val_loss: 6.7458
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0456 - loss: 6.3177 - val_accuracy: 0.0521 - val_loss: 6.7954
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0511 - loss: 6.1715 - val_accuracy: 0.0550 - val_loss: 6.8197
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0549 - loss: 5.9947 - val_accuracy: 0.0616 - val_loss: 6.8909
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0666 - loss: 5.8526 - val_accuracy: 0.0653 - val_loss: 6.9296
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0796 - loss: 5.6498 - val_accuracy: 0.0645 - val_loss: 6.9360
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - accuracy: 0.0826 - loss: 5.5121 - val_

In [50]:
def predict(model, tokenizer, text, max_sequence_length):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-max_sequence_length+1:]
    token_list = pad_sequences(
        [token_list],
        maxlen=max_sequence_length-1,
        padding='pre',
    )
    predicted = model.predict(token_list, verbose=0)
    preicted_word_index = np.argmax(predicted, axis = 1)
    for word, index in tokenizer.word_index.items():
        if index == preicted_word_index:
            return word
    
    return None

In [51]:
predict(
    model,
    tokenizer=tokenizer,
    max_sequence_length=max_sequence_length,
    text="to be or not to be",
)

'of'

In [52]:
model.save("model.h5")
import pickle
with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)